In [ ]:
import os

from src.aggregates import ofb_per_subtype_per90min
from src.load_data import load_matches


In [15]:
# load matches
matches_json_path = os.path.join(os.getcwd(), "src/data/matches.json")
all_tracking, dynamic_events_all, all_metadata = load_matches(matches_json_path)
print(dynamic_events_all.head())
print(dynamic_events_all["targeted"].dtype)
print(dynamic_events_all["targeted"].head())

# compute off-ball event stats per subtype per90min
ofb_stats_per90min = ofb_per_subtype_per90min(all_tracking, dynamic_events_all)

print(ofb_stats_per90min)

c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\load_data.py:54: DtypeWarning: Columns (75,77,184,264) have mixed types. Specify dtype option on import or set low_memory=False.
  de_match = pd.read_csv(url)
c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\load_data.py:54: DtypeWarning: Columns (276) have mixed types. Specify dtype option on import or set low_memory=False.
  de_match = pd.read_csv(url)


  event_id  index  match_id  frame_start  frame_end  frame_physical_start  \
0      8_0      0   2017461         2512       2512                   NaN   
1      1_0      1   2017461         2523       2539                   NaN   
2      8_1      2   2017461         2526       2540                   NaN   
3      7_0      3   2017461         2526       2528                   NaN   
4      7_1      4   2017461         2526       2540                   NaN   

  time_start time_end  minute_start  second_start  ...  \
0    00:00.2  00:00.2             0             0  ...   
1    00:01.3  00:02.9             0             1  ...   
2    00:01.6  00:03.0             0             1  ...   
3    00:01.6  00:01.8             0             1  ...   
4    00:01.6  00:03.0             0             1  ...   

   xloss_player_possession_end  xloss_player_possession_max  \
0                          NaN                          NaN   
1                          NaN                          NaN   

KeyError: 'targeted'